### Import necessary tools

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc

import torch
from torch.utils.data import TensorDataset

In [2]:
NUM_CLASSES = 919

### Load the test set and the trained model

In [4]:
net = torch.load('../models/neuralsea-epoch-27-loss-0.0527.pth', map_location=torch.device('cpu'))
net.eval()

test_set = TensorDataset(torch.from_numpy(np.load('../data/X_test.npy')),
                         torch.from_numpy(np.load('../data/y_test.npy')))

In [5]:
y_test = test_set[:1000][1].float()
y_pred = net(test_set[:1000][0].float())

AttributeError: 'LSTM' object has no attribute '_flat_weights'

# Compute ROC curve and ROC area for each class

In [ ]:
fpr = {}
tpr = {}
roc_auc = {}
for i in range(NUM_CLASSES):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area

In [ ]:
fpr['micro'], tpr['micro'], _ = roc_curve(y_test.ravel(), y_pred.ravel())
roc_auc['micro'] = auc(fpr['micro'], tpr['micro'])

# Plot ROC curves

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[0], tpr[0], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()

# Compute macro-average ROC curve and ROC area

In [ ]:
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(NUM_CLASSES)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(NUM_CLASSES):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= NUM_CLASSES

fpr['macro'] = all_fpr
tpr['macro'] = mean_tpr
roc_auc['macro'] = auc(fpr['macro'], tpr['macro'])

# Plot micro and macro ROC curves

In [ ]:
plt.figure()
lw = 4
plt.plot(fpr['micro'], tpr['micro'],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc['micro']),
         color='deeppink', linestyle=':', lw=lw)

plt.plot(fpr['macro'], tpr['macro'],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc['macro']),
         color='navy', linestyle=':', lw=lw)

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc='lower right')
plt.show()